# Modelado de sistemas eléctricos

## Un circuito sencillo

Empezaremos esta práctica modelando un sistema sencillo, un circuito con una resistencia y una capacitancia como el de la siguiente figura:

![](./imagenes/cir1.png)

Este circuito tiene una ecuación diferencial:

$$
V_1 = I R_1 + \frac{1}{C_1} \int I dt
$$

en donde $R_1$ representa el valor de la resistencia ```R1```, y la caida de voltaje en esta resistencia es $I R_1$; $C_1$ representa el valor del capacitor ```C1```, y la caida de voltaje en este capacitor es $\frac{1}{C_1}\int I dt$; por lo que si sumamos las fuentes de voltaje y las igualamos a las caidas de voltaje (todos los elementos que consumen una parte de este voltaje) podremos obtener esta ecuación.

Si ahora hacemos increiblemente obvio cuales de estas cantidades son variables y cuales son constantes, llegaremos a escribir esta ecuación como:

$$
V_1(t) = I(t) R_1 + \frac{1}{C_1} \int I(t) dt
$$

En donde si considero que el voltaje suministrado por la fuente de alimentación puede variar con lo que pasa el tiempo, llego a la conclusión que el voltaje varia con el tiempo, es decir

$$
V_1 = V_1(t)
$$

Y que cuando este voltaje varíe, la corriente a traves del circuito cambiará, por lo que:

$$
I = I(t)
$$

La herramienta que utilizaremos para obtener una simulación de este sistema será la transformada de Laplace; sin embargo, para este tipo de sistemas podemos utilizar una simplificación de estas tomando en cuenta que solo servirán si mi sistema empieza con condiciones iniciales $0$ (en este caso significa que nuestro sistema empieza apagado, y en el tiempo $t=0$ lo enciendo):

$$
\begin{align}
\mathcal{L}\left\{ \int f(t) dt \right\} &= \frac{1}{s} F(s) \\
\mathcal{L}\left\{ f(t) \right\} &= F(s) \\
\mathcal{L}\left\{ f'(t) \right\} &= sF(s) \\
\mathcal{L}\left\{ f''(t) \right\} &= s^2F(s) \\
\end{align}
$$

Si aplicamos estas transformadas, obtendremos:

$$
V_1(s)  = I(s) R_1 + \frac{1}{C_1} \frac{1}{s} I(s)
$$

El siguiente paso será obtener la función de transferencia del sistema, para esto tenemos que pensar en el sistema como una caja negra, de la cual queremos obtener una relación:

![](./imagenes/sis1.png)

Si pensamos en este sistema como una caja que multiplica su entrada por lo que sea que tenga adentro y como resultado de esta multiplicación obtenemos la salida, la ecuación de lo que tiene adentro tiene que ser:

$$
G(s) = \frac{Salida}{Entrada}
$$

por lo que nuestro objetivo tiene que ser obtener esta ecuación. Como dijimos anteriormente, la corriente $I(t)$ es el resultado de conectar nuestro circuito a un voltaje, por lo que nuestra función de transferencia tiene que ser:

$$
G(s) = \frac{I(s)}{V(s)}
$$

Si ahora despejamos $V(s)$ por un lado, y $I(s)$ del otro, obtendremos:

$$
V(s) = I(s)\left[ R_1 + \frac{1}{C_1s} \right]
$$

Obteniendo la división de $I(s)$ entre $V(s)$, obtendremos:

$$
\frac{I(s)}{V(s)} = \frac{1}{R_1 + \frac{1}{C_1s}}
$$

Si ahora multiplicamos tanto el numerador, como el denomidador por $s$, obtendremos:

$$
\frac{I(s)}{V(s)} = G_1(s) = \frac{s}{R_1 s + \frac{1}{C_1}}
$$

Con esta función de transferencia podremos simular nuestro sistema, para esto importamos las funciones ```tf``` y ```step_response``` de la librería de control, así como ```linspace``` de ```numpy```:

In [ ]:
from control import tf, step_response
from numpy import linspace

Para definir la función de transferencia debemos usar la función ```tf``` escribiendo el numerador de la función $G(s)$ primero y el denominador despues:

$s = s + 0\implies$ ```[1, 0]```

$R_1 s + \frac{1}{C_1} \implies$ ```[R1, 1/C1]```

In [ ]:
R1 = 220
C1 = 0.1e-3
G1 = tf([1, 0], [R1, 1/C1])

Usamos ```linspace``` para definir los limites de tiempo de nuestra simulación, así como el número de puntos a utilizar; en este caso simularemos nuestro sistema desde el tiempo $0s$ hasta el tiempo $0.4s$ con un total de $100$ puntos:

In [ ]:
ts = linspace(0, 0.4, 100)

Una vez que hemos definido todos estos parametros, los metemos a ```step_response``` para obtener nuestros datos:

In [ ]:
t, y = step_response(G1, ts)

Y graficarlos con la función ```plot``` de ```matplotlib```:

In [ ]:
from matplotlib.pyplot import plot

In [ ]:
plot(t, y);

Por lo que podemos ver que el comportamiento del sistema es tener un pico de corriente, en cuanto es conectado, de un poco mas de $4mA$, sin embargo este es el comportamiento del sistema cuando se le conecta un voltaje de $1V$, debido a que por default la función ```step_response``` simulará el comportamiento del sistema ante un escalon unitario.

Para poder simular el sistema ante un escalon de $5V$, tendremos que premultiplicar este sistema por una amplificación de $5$:

In [ ]:
A = 5

In [ ]:
t, y = step_response(A*G1, ts)

In [ ]:
plot(t, y);

Por lo que sabemos que al inicio de la simulación se tiene un pico de un poco mas de $20mA$.

Una vez que tenemos la simulación de la corriente del sistema, podemos obtener el voltaje en cualquier elemento, por ejemplo en el capacitor; para hacer esto necesitamos la función de transferencia del capacitor, la cual transforma una corriente $I(t)$ en un voltaje $V_C(t)$, si recordamos, ya tenemos esta ecuación:

$$
V_C = \frac{1}{C_1} \int I dt \implies \frac{V_C(s)}{I(t)} = G_2(s) = \frac{1}{C_1} \frac{1}{s} = \frac{1}{C_1s}
$$

por lo que definimos esta función de transferencia:

In [ ]:
G2 = tf([1], [C1, 0])

Y simulamos el comportamiento de los dos sistemas uno detras de otro:

In [ ]:
t, y = step_response(A*G1*G2, ts)

In [ ]:
plot(t, y);

En esta gráfica lo que podemos ver es el capacitor llenandose con el voltaje aplicado en aproximadamente una decima de segundo.

---
## Ejercicio

Tomando en cuenta el siguiente circuito:

![](./imagenes/cir2.png)

con la siguiente ecuación diferencial:

$$
V_1 = L_1 \frac{dI}{dt} + I R_1 + \frac{1}{C_1} \int I dt
$$

* Define una función de transferencia $G_3$ y guardala en la variable ```G3```

> Nota: Recuerda eliminar la linea ```raise NotImplementedError``` cuando respondas cada ejercicio

In [ ]:
L1 = 0.01
# ESCRIBE TU CODIGO AQUI
raise NotImplementedError

In [ ]:
from numpy.testing import assert_allclose

assert_allclose(G3.pole(), [-21954.4511501,    -45.5488499])
assert_allclose(G3.zero(), [0])

* Simula el comportamiento de la corriente $I(t)$ cuando se le es aplicado un voltaje de $12V$, desde el tiempo $0s$ hasta el tiempo $1s$

In [ ]:
# ESCRIBE TU CODIGO AQUI
raise NotImplementedError
plot(t, y);

In [ ]:
from nose.tools import assert_almost_equal, assert_equal

assert_equal(ts[0], 0)
assert_equal(ts[-1], 1)
assert_almost_equal(max(y), 0.034573738498078065)
assert_almost_equal(y[-1], 0)

* Simula el comportamiento del voltaje en el inductor $V_L$

In [ ]:
# ESCRIBE TU CODIGO AQUI
raise NotImplementedError
plot(t, y);

In [ ]:
from nose.tools import assert_almost_equal, assert_equal

assert_equal(ts[0], 0)
assert_equal(ts[-1], 1)
assert_almost_equal(max(y), 12)
assert_almost_equal(y[1], -0.01574794025216697)
assert_almost_equal(y[-1], 0)

---

## Un circuito complejo

Si ahora queremos simular un sistema mas complejo, tenemos que utilizar un truco muy simple, pero antes analicemos como logramos simular el voltaje en el ejemplo pasado.

En el ejemplo anterior ya teniamos una función de transferencia para el sistema completo, pero queriamos especificamente el voltaje, no la corriente, por lo que creamos un sistema nuevo y los simulamos uno detras de otro, esto es equivalente al siguiente diagrama:

![](./imagenes/sis2.png)

en donde podemos anotar las señales:

![](./imagenes/sis3.png)

por lo que podemos pensar en la multiplicación de los dos sitemas como un sistema nuevo, podemos revisar esto si analizamos las variables guardadas:

In [ ]:
G1

In [ ]:
G2

In [ ]:
G1*G2

Nota que las funciones de transferencia se multiplican correctamente cuando se lo indicamos.

Si ahora revisamos nuestro circuito complejo:

![](./imagenes/cir3.png)

podriamos utilizar Leyes de Kirchoff para obtener ecuaciones diferenciales de este sistema y sería completamente valido, sin embargo, podemos tambien dividir este problema en varios pequeños.

El lazo de la izquierda es equivalente a la función de transferencia $G_1$ que ya obtuvimos, mas aún, si ahora lo multiplicamos por la función de transferencia $G_2$ estaremos obteniendo la función de transferencia que haria esto equivalente a una fuente de alimentación de voltaje variable, es decir:

![](./imagenes/cir4.png)

por lo que ahora solo tenemos que obtener otra función de transferencia para el sistema nuevo.

In [ ]:
R2 = 1000
C2 = 100e-9
L2 = 0.1
G1 = tf([1, 0], [L1, R1, 1/C1])
G2 = tf([1], [C1, 0])
G3 = tf([1, 0], [L2, R2, 1/C2])

y simularlo todo completo:

In [ ]:
t, y = step_response(5*G1*G2*G3, ts)

In [ ]:
plot(t, y);

y para obtener el voltaje en el capacitor $C_2$

In [ ]:
G4 = tf([1], [C2, 0])

In [ ]:
t, y = step_response(5*G1*G2*G3*G4, ts)
plot(t, y);

---
## Ejercicio

* Simule el comportamiento del voltaje en el capacitor $C_2$ del siguiente circuito del tiempo $0s$ a $0.2s$ con 1000 puntos:

![](./imagenes/cir5.png)

> Nota: Utiliza las variables ```G1```, ```G2```, ```G3```, ```G4```, ```G5``` y ```G6```

In [ ]:
V1 = 12
L1 = 0.01
L2 = 0.001
R1 = 4700
R2 = 220
C1 = 0.47e-3
C2 = 100e-9

# ESCRIBE TU CODIGO AQUI
raise NotImplementedError
plot(t, y);

In [ ]:
from nose.tools import assert_almost_equal, assert_equal
from numpy.testing import assert_allclose

assert_equal(ts[0], 0)
assert_equal(ts[-1], 0.2)
assert_equal(len(ts), 1000)

assert_almost_equal(max(y), 24, 2)
assert_almost_equal(min(y), 0, 2)
assert_almost_equal(y[0], 0)

assert_allclose((V1*G1*G2*G3*G4*G5*G6).zero(), [0, 0])
assert_allclose((V1*G1*G2*G3*G4*G5*G6).pole(), [-4.7e+06+0.j, -4.5e+04+0.j, 1.5e-12+461.2j,
                                                1.5e-12-461.2j,0.0e0 +0.j, 0.0e0+0.j], 5)

* Defina una función de transferencia ```G7``` que sea capaz de simular el comportamiento de un amplificador operacional, el cual amplifica su señal de entrada $10$ veces.

In [ ]:
# ESCRIBE TU CODIGO AQUI
raise NotImplementedError

In [ ]:
from numpy.testing import assert_allclose

assert_allclose(G7.pole(), [])
assert_allclose(G7.zero(), [])
assert_allclose(G7.dcgain(), [10])

* Defina una función de transferencia ```G8``` que sea capaz de simular el comportamiento de un amplificador operacional configurado como un amplificador integrador, con una ganacia de $5$

In [ ]:
# ESCRIBE TU CODIGO AQUI
raise NotImplementedError

In [ ]:
from numpy.testing import assert_allclose
from numpy import inf

assert_allclose(G8.pole(), [0])
assert_allclose(G8.zero(), [])
assert_allclose(G8.dcgain(), [inf])

---